# Using DSP to improve the accuracy of an EEG classifier

**Authors**
1. Kasra Lekan (kl5sq)
2. Derek Johnson (dej3tc)
3. Fiji Marcelin (fm4cg)

## Experimental Setup
**Signal Data**: For now, we plan on applying these techniques to sleep stage EEG data. This data is taken from a study looking at “slow-wave microconfinuity” during sleep [1]. 

**MNE Package**: The MNE package ...

## Method 1: Peak Finder
### Theory
    
### Implementation


In [ ]:
# import peak_finder as pf

@Derek and @Fiji

## Experiment Results

## Challenges
...

## Work Breakdown

Kasra Lekan: 
- Experimental setup and background MNE research
- Peak Detection Algorithm

Derek Johnson: 
- Butterworth Filter

Fiji Marcelin: 
- Band Pass Filter

All:
- Combining filters and testing classification accuracy

## References
1. B. Kemp, A. H. Zwinderman, B. Tuk, H. A. C. Kamphuisen, and J. J. L. Oberyé. Analysis of a sleep-dependent neuronal feedback loop: the slow-wave microcontinuity of the EEG. IEEE Transactions on Biomedical Engineering, 47(9):1185–1194, 2000. doi:10.1109/10.867928.
    Dataset for analysis
2. https://mne.tools/stable/index.html
    Implementation of foundational EEG signals pipleline
3. https://neuraldatascience.io/intro.html
    E-book that covers analysis of EEG data in the frequency domain.
4. https://en.wikipedia.org/wiki/Band-pass_filter
5. https://en.wikipedia.org/wiki/Butterworth_filter
6. https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html
7. http://www.scholarpedia.org/article/Electroencephalogram
    An overview of electroencephologram (EEG) collection. 
